#### Descrição

> Notebook responsável por gerar tabelas CSVs pedido e item_pedido

##### Generating CSV Data and DF for Pedido and Item_Pedido

In [2]:
import pandas as pd
import csv
from random import shuffle, choice, randrange, random
from faker import Faker
from datetime import datetime, timedelta
import locale
import os

fake = Faker()

def gen_datetime(min_year=1985, max_year=datetime.now().year):
    # generate a datetime in format yyyy-mm-dd hh:mm:ss.000000
    start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=365 * years)
    return start + (end - start) * random() 

tbl_id = [id for id in range(1, 8)]
id_unidade = tbl_id.copy()
vlr_pedido = [111, 122, 225, 1008, 409, 887, 77]
vlr_item = [2.87, 88.77, 77.65, 10.13, 14.8, 9.88, 1.22]
shuffle(id_unidade)
shuffle(vlr_pedido)
shuffle(vlr_item)

pedido = {
        "Id_Unidade": id_unidade, 
        "Id_Pedido":  tbl_id, 
        "Tipo_Pedido": [choice((1, 2)) for tipo_p in range(1, 8)],
        "Data_Pedido": [gen_datetime().strftime("%Y/%m/%d") for data_p in range(1, 8)],
        "Vlr_Pedido": vlr_pedido,
        "Endereco_Entrega": [str(fake.address()) for endereco in range(1, 8)],
        "Taxa_Entrega": [randrange(1, 10) * 2.75 for taxa_entrega in range(1, 8)],
        "Status": [choice((1, 2, 3)) for st in range(1, 8)]
        }

id_pedido = id_unidade.copy()
shuffle(id_pedido)
id_produto = id_pedido.copy()
shuffle(id_produto)

item_pedido = { 
        "Id_Pedido":  id_pedido, 
        "Id_Item_Pedido": tbl_id,
        "Id_Produto": id_produto,
        "Qtd": [choice([i for i in range(100)]) for qtd in range(1, 8)],
        "Vlr_Item": vlr_item,
        "Observacao": [None for taxa_entrega in range(1, 8)],
        }


df_pedido = pd.DataFrame(pedido)
df_item_pedido = pd.DataFrame(item_pedido)

df_pedido = df_pedido.astype({'Data_Pedido':'datetime64[ns]'})
df_pedido = df_pedido.astype({'Endereco_Entrega':'string'})
df_item_pedido = df_item_pedido.astype({'Observacao':'string'})

print("Pedido DF: ")
display(df_pedido)
display(df_pedido.dtypes)

print("Item Pedido DF: ")
display(df_item_pedido)
display(df_item_pedido.dtypes)

Pedido DF: 


,Id_Unidade,Id_Pedido,Tipo_Pedido,Data_Pedido,Vlr_Pedido,Endereco_Entrega,Taxa_Entrega,Status
0,4,1,2,1997-07-11,887,"69086 Snyder Brooks Apt. 681 Clarkeport, GU 54188",19.25,1
1,1,2,1,2014-02-14,122,"88672 Wright Inlet Apt. 231 Smithville, WA 96406",2.75,1
2,2,3,1,1993-10-04,409,"7698 Diaz Ramp North Virginia, NM 61412",2.75,2
3,5,4,2,1988-05-02,77,"3399 Weeks Tunnel Lopezberg, MI 11309",22.00,1
4,6,5,1,1991-05-21,111,92461 James Gateway Suite 708 Lake Christiansi...,19.25,2
5,3,6,2,2012-11-21,1008,"049 Natasha Wells Walkerside, WY 20533",19.25,2
6,7,7,1,2020-06-27,225,"4046 Blevins Mountains East Arianaberg, MA 26968",5.50,2


Id_Unidade                   int64
Id_Pedido                    int64
Tipo_Pedido                  int64
Data_Pedido         datetime64[ns]
Vlr_Pedido                   int64
Endereco_Entrega    string[python]
Taxa_Entrega               float64
Status                       int64
dtype: object

Item Pedido DF: 


,Id_Pedido,Id_Item_Pedido,Id_Produto,Qtd,Vlr_Item,Observacao
0,2,1,1,70,1.22,<NA>
1,6,2,3,45,9.88,<NA>
2,5,3,4,3,2.87,<NA>
3,1,4,7,80,10.13,<NA>
4,3,5,5,20,88.77,<NA>
5,7,6,2,75,77.65,<NA>
6,4,7,6,49,14.80,<NA>


Id_Pedido                  int64
Id_Item_Pedido             int64
Id_Produto                 int64
Qtd                        int64
Vlr_Item                 float64
Observacao        string[python]
dtype: object

##### Creation of CSVs

In [5]:
def pivot_table(table_values):
    values, temp_list = [], []
    for idx in range(len(list(table_values.values())[0])):
        for value in table_values.values():
            temp_list += [value[idx]]
        values.append(temp_list)
        temp_list = []
    return values

pedido_values, item_pedido_values = pivot_table(pedido), pivot_table(item_pedido)


def create_csv(name, table, table_values):
    with open(f"./csv-tables/{name}.csv", 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(table)
    
        for row in table_values:
            writer.writerow(row)

try:
    create_csv("pedido", list(pedido.keys()), pedido_values)
    create_csv("item_pedido", list(item_pedido.keys()), item_pedido_values)
except Exception as e:
    print(f"Erro: {e}. CSVs não criados.")
else:
    print("CSVs pedido e item_pedido criados.")

CSVs pedido e item_pedido criados.
